- Usage
    - Basic
    - Text prompts
    - Message prompts
    - Dictionary format
- Message types
    - System Message
    - Human Message
    - AI Message, 
    - Tool Message
    - Streaming and chunks
- Content
    - Standard content blocks
    - Multimodal
- Examples


### 기본 사용법 (Basic Usage)
메시지 객체를 생성하고 호출할 때 모델에 전달

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("openai:gpt-5-nano")

system_msg = SystemMessage("You are a helpful assistant.")
human_msg = HumanMessage("Hello, how are you?")

# Use with chat models
messages = [system_msg, human_msg]
response = model.invoke(messages)  # Returns AIMessage

- 텍트스 프롬프트 (Text prompts)
문자열이므로 대화 기록을 보관할 필요가 없는 간단한 생성 작업에 적합.

In [ ]:
response = model.invoke("Write a haiku about spring")

- 메시지 프롬프트 (Message prompts)
메시지 객체 목록을 제공하여 모델에 메시지 목록을 전달.
여러 차례의 대화 관리, 다중 모드 콘텐츠(오디오, 이미지, 파일) 작업, 시스템 지침 포함 등에 사용

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage("You are a poetry expert"),
    HumanMessage("Write a haiku about spring"),
    AIMessage("Cherry blossoms bloom...")
]
response = model.invoke(messages)

- 사전 형식 (Dictionary prompts)
OpenAI 채팅 완성 형식으로 메시지를 직접 지정할 수도 있음.

In [ ]:
messages = [
    {"role": "system", "content": "You are a poetry expert"},
    {"role": "user", "content": "Write a haiku about spring"},
    {"role": "assistant", "content": "Cherry blossoms bloom..."}
]
response = model.invoke(messages)

### Message types
 시스템 메시지 - 모델이 상호 작용에 대한 컨텍스트를 제공하고 동작하는 방법을 알려줍니다.
 인간 메시지 - 사용자 입력 및 모델과의 상호 작용을 나타냅니다.
 AI 메시지 - 텍스트 콘텐츠, 도구 호출, 메타데이터를 포함한 모델에서 생성된 응답
 도구 메시지 - 도구 호출의 출력을 나타냅니다.

- 시스템 메시지 (System Message)
모델의 행동을 준비하는 초기 지침을 나타냄. 시스템 메시지를 사용하여 분위기를 설정하고, 모델의 역할을 정의하고, 응답 지침을 설정할 수 있음.

In [ ]:
system_msg = SystemMessage("You are a helpful coding assistant.")

messages = [
    system_msg,
    HumanMessage("How do I create a REST API?")
]
response = model.invoke(messages)

# 좀 더 자세한 지침
from langchain_core.messages import SystemMessage, HumanMessage

system_msg = SystemMessage("""
You are a senior Python developer with expertise in web frameworks.
Always provide code examples and explain your reasoning.
Be concise but thorough in your explanations.
""")

messages = [
    system_msg,
    HumanMessage("How do I create a REST API?")
]
response = model.invoke(messages)

- 인간의 메시지 (Human Message)
사용자 입력과 상호작용을 나타냄. 텍스트, 이미지, 오디오, 파일 및 기타 다양한 멀티모달 콘텐츠를 포함할 수 있음.

In [ ]:
human_msg = HumanMessage("What is machine learning?")
response = model.invoke([human_msg])

# 메시지 메타데이터
human_msg = HumanMessage(
    content="Hello!",
    name="alice",  # Optional: identify different users
    id="msg_123",  # Optional: unique identifier for tracing
)

- AI 메시지 (AI Message)
모델 호출의 출력을 나타냄. 여기에는 나중에 엑세스할 수 있는 다중 모드 데이터, 도구 호출 및 공급자별 메타데이터가 포함될 수 있음.

In [ ]:
response = model.invoke("Explain AI")
print(type(response))  # <class 'langchain_core.messages.AIMessage'>

# 메뉴얼 AI 응답 메시지
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage

# Create an AI message manually (e.g., for conversation history)
ai_msg = AIMessage("I'd be happy to help you with that question!")

# Add to conversation history
messages = [
    SystemMessage("You are a helpful assistant"),
    HumanMessage("Can you help me?"),
    ai_msg,  # Insert as if it came from the model
    HumanMessage("Great! What's 2+2?")
]

response = model.invoke(messages)

- 도구 호출 응답 (Tool Message)

In [ ]:
model_with_tools = model.bind_tools([...])
response = model_with_tools.invoke("What's the weather in Paris?")

for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")
    print(f"ID: {tool_call['id']}")


도구 호출을 지원하는 모델의 경우, AI 메시지에 도구 호출이 포함될 수 있음. 도구 메시지는 단일 도구 실행 결과를 모델로 다시 전달하는데 사용됨.

In [ ]:
from langchain_core.messages import ToolMessage
# After a model makes a tool call
ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "San Francisco"},
        "id": "call_123"
    }]
)

# Execute tool and create result message
weather_result = "Sunny, 72°F"
tool_message = ToolMessage(
    content=weather_result,
    tool_call_id="call_123"  # Must match the call ID
)

# Continue conversation
messages = [
    HumanMessage("What's the weather in San Francisco?"),
    ai_message,  # Model's tool call
    tool_message,  # Tool execution result
]
response = model.invoke(messages)  # Model processes the result

- 스트리밍 및 청크 (Streaming and chunks)
스트리밍하는 동안 결합할 수 있는 AIMessageChunk 객체를 받게 됨.

In [ ]:
chunks = []
for chunk in model.stream("Write a poem"):
    chunks.append(chunk)
    print(chunk.text)

### 콘텐츠 (Content)
메지니의 내용은 모델로 전송되는 데이터 페이로드라고 생각하면 됨. 메시지는 느슨한 타입의 속성을 가지며, 문자열과 타입이 지정되지 않은 객체 목록(예: Dictionary)를 지원함. 

In [ ]:
# 다중 입력 모드. 
from langchain_core.messages import HumanMessage

# String content
human_message = HumanMessage("Hello, how are you?")

# Provider-native format (e.g., OpenAI)
human_message = HumanMessage(content=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
])

# List of standard content blocks
human_message = HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://example.com/image.jpg"},
])

- 표준 메시지 블록 (Standard content blocks)

In [ ]:
# OpenAI
from langchain_core.messages import AIMessage

message = AIMessage(
    content=[
        {
            "type": "reasoning",
            "id": "rs_abc123",
            "summary": [
                {"type": "summary_text", "text": "summary 1"},
                {"type": "summary_text", "text": "summary 2"},
            ],
        },
        {"type": "text", "text": "...", "id": "msg_abc123"},
    ],
    response_metadata={"model_provider": "openai"}
)
message.content_blocks
# [{'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 1'},
#  {'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 2'},
#  {'type': 'text', 'text': '...', 'id': 'msg_abc123'}]

# ---
# Anthropic
from langchain_core.messages import AIMessage

message = AIMessage(
    content=[
        {"type": "thinking", "thinking": "...", "signature": "WaUjzkyp..."},
        {"type": "text", "text": "..."},
    ],
    response_metadata={"model_provider": "anthropic"}
)
message.content_blocks
# [{'type': 'reasoning',
#   'reasoning': '...',
#   'extras': {'signature': 'WaUjzkyp...'}},
#  {'type': 'text', 'text': '...'}]

- 멀티모달 (Multimodal)

In [ ]:
# From URL
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "url": "https://example.com/path/to/image.jpg"},
    ]
}

# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {
            "type": "image",
            "base64": "AAAAIGZ0eXBtcDQyAAAAAGlzb21tcDQyAAACAGlzb2...",
            "mime_type": "image/jpeg",
        },
    ]
}

# From provider-managed File ID
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "file_id": "file-abc123"},
    ]
}

### Example
대화형 애플리케이션을 구축하려면 메시지 기록과 컨텍스트를 관리해야 함.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

# Initialize conversation
messages = [
    SystemMessage("You are a helpful assistant specializing in Python programming")
]

# Simulate multi-turn conversation
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break

    # Add user message
    messages.append(HumanMessage(user_input))

    # Get model response
    response = model.invoke(messages)

    # Add assistant response to history
    messages.append(response)

    print(f"Assistant: {response.content}")